In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle 
pd.set_option('display.max_columns', None)

In [2]:
# Load in our data
big_df = pd.read_csv("../data/big_df.csv")
big_df = big_df.drop(columns="lev1")
big_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (215,216) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,-1.245766,13.853492,4.117214,15.900697,5.362980,2.047206,1991-02-21,1988-09-29,1.028807,0.487211,1,0,0.743590,0.664234,0.250000,0.366013,0.750000,0.744000,0.250000,0.350877,0.359700,0.513778,0.528960,0.622201,0.677419,0.688073,0.333333,0.000000,0.313528,0.470707,0.581265

In [3]:
# removing fights that have not taken place yet
df = big_df[big_df["winner"] != "tbd"]
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [4]:
# Dropping all fights that occured before unified rules were implamented 
df["date"] = pd.to_datetime(df["date"])
df = df[df["date"] >= "November 17, 2000"]
df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [5]:
df["red_age"] = pd.Series(df["red_age"], dtype="int")
df["blue_age"] = pd.Series(df["blue_age"], dtype="int")
df["red_heightIN"] = pd.Series(df["red_heightIN"], dtype="int")
df["blue_heightIN"] = pd.Series(df["blue_heightIN"], dtype="int")
df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [6]:
df = df[df["red_age"] < 75]
df = df[df["blue_age"] < 75]
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [7]:
df = df[df["red_heightIN"] < 100]
df = df[df["blue_heightIN"] < 100]
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [8]:
# Label encoder for stance and weight class
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
df['red_stance'] = number.fit_transform(df['red_stance'].astype('str'))
df['blue_stance'] = number.fit_transform(df['blue_stance'].astype('str'))
df['weight_class'] = number.fit_transform(df['weight_class'].astype('str'))
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [9]:
# removing draws and no contests 
df = df[df["winner"] != "Draw"] # it's arguable draws are useful datapoints, but I choose to remove them
df = df[df["winner"] != "NC"] # no contests however are not useful datapoints
df["winner"].value_counts()

red     3827
blue    2016
Name: winner, dtype: int64

In [10]:
df = df[df["method"] != "DQ"]
df["method"].value_counts()

UD            2135
KO/TKO        1938
Submission    1138
SD             571
MD              43
Name: method, dtype: int64

In [11]:
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [12]:
# setting our X and y values
X = df.drop(columns=["winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method", "location", \
                    "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"])
y = df["winner"]
feature_names = X.columns

In [13]:
# illustrate the concept of resampling with a logistic regression model before applying it to our random forest
clf_0 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_0 = clf_0.predict(X)
pred_y_0

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['blue', 'red', 'red', ..., 'red', 'red', 'red'], dtype=object)

In [14]:
# How's the accuracy?
print(accuracy_score(pred_y_0, y))

0.6643776824034334


In [15]:
print( np.unique( pred_y_0 ) )

['blue' 'red']


In [16]:
df["winner"].value_counts()

red     3815
blue    2010
Name: winner, dtype: int64

In [17]:
# Separate majority and minority classes
df_majority = df[df.winner=="red"]
df_minority = df[df.winner=="blue"]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority), # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.winner.value_counts()

red     3815
blue    3815
Name: winner, dtype: int64

In [18]:
df_upsampled

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
14,7.203718,-19.255047,16.731216,3.798692,9.527498,23.053739,1991-08-24,1988-07-26,2.666667,0.932836,0,0,0.857143,0.632911,0.105263,0.279570,0.708333,0.722892,0.240000,0.330709,0.366412,0.451874,0.580488,0.510504,0.770833,0.731707,0.217391,0.189369,0.411184,0.417431,0.5569

In [19]:
# Separate input features (X) and target variable (y)
y = df_upsampled.winner
X = df_upsampled.drop(columns=["winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method", \
                               "location", "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"], axis=1)
 

feature_names = X.columns
# Train model
clf_1 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_1 = clf_1.predict(X)
 
# Is our model still predicting just one class?
print( np.unique( pred_y_1 ) )

['blue' 'red']


C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
# How's our accuracy?
print(accuracy_score(y, pred_y_1))

0.6107470511140236


In [21]:
# Now that we can see resampling will help us increase accuracy on a logistic regression, lets see on random forest
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [22]:
# our decision tree score has increased dramatically
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7520964360587002

In [23]:
sorted(zip(clf.feature_importances_, X), reverse=True)

[(0.05120698427370908, 'red_StkHeadAbsPerMin'),
 (0.028810757234916634, 'red_age'),
 (0.023127429513990078, 'red_SigDistStkDef'),
 (0.018888406010423416, 'red_winPCT'),
 (0.01887914864282669, 'red_StkBodyPerMin'),
 (0.018214258698029975, 'blue_reachIN'),
 (0.01735500950759074, 'blue_age'),
 (0.015705371635216896, 'red_sumSigBodyAtt'),
 (0.01472848445132443, 'blue_StkBodyPerMin'),
 (0.01461199567392471, 'blue_TotStkDef'),
 (0.013790874605773284, 'red_StkLegPerMin'),
 (0.013255871036748528, 'red_heightIN'),
 (0.011939395225906537, 'red_TotStkDef'),
 (0.01185144640420335, 'red_sumSigBodyAbs'),
 (0.011522965236039214, 'red_SigStkAcc'),
 (0.011458791163684727, 'red_TotStkPerMinDiff'),
 (0.011161284720558586, 'blue_StkLegPerMin'),
 (0.010767344174036794, 'red_SigGndStkDef'),
 (0.010556805880367915, 'red_SoS'),
 (0.010315515064731313, 'red_reachIN'),
 (0.009871117815068057, 'weight_class'),
 (0.009684072396762732, 'blue_heightIN'),
 (0.009406173828734118, 'red_SigBodyStkAcc'),
 (0.00935048821

In [24]:
# and so has our random forest
rf = RandomForestClassifier(n_estimators=105)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8149895178197065

In [25]:
preds = rf.predict(X)

In [26]:
filename = "../saved_models/rf_up_model.sav"
pickle.dump(rf, open(filename, 'wb'))

In [27]:
# Repeating proccess by down sampling red wins

In [28]:
# Separate majority and minority classes
df_majority = df[df.winner=="red"]
df_minority = df[df.winner=="blue"]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority), # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.winner.value_counts()

blue    2010
red     2010
Name: winner, dtype: int64

In [29]:
# illustrating this concept with a logistic regression model before applying it to our random forest

In [30]:
# Separate input features (X) and target variable (y)
y = df_downsampled.winner
X = df_downsampled.drop(columns=["winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method", \
                               "location", "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"], axis=1)
 
# Train model
clf_2 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_2 = clf_2.predict(X)
 
# Is our model still predicting just one class?
pred_y_2

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['red', 'red', 'red', ..., 'red', 'red', 'blue'], dtype=object)

In [31]:
# How's our accuracy?
print( accuracy_score(y, pred_y_2) )

0.6106965174129353


In [32]:
# repeating on down-sampled data

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [34]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5402985074626866

In [35]:
sorted(zip(clf.feature_importances_, X), reverse=True)

[(0.04431691580425647, 'red_StkHeadAbsPerMin'),
 (0.0298364494217864, 'red_age'),
 (0.022573110268213535, 'blue_reachIN'),
 (0.021424673510334703, 'red_TDAcc'),
 (0.01748139248252399, 'blue_age'),
 (0.016959545430343375, 'red_SigStkAbsPerMin'),
 (0.016559838066731745, 'red_SigHeadStkAcc'),
 (0.01640493595534846, 'blue_SigBodyStkDef'),
 (0.01589293196326572, 'red_reachIN'),
 (0.015192509685914934, 'blue_heightIN'),
 (0.014977848306240561, 'red_StkBodyPerMin'),
 (0.014879436000533512, 'red_SigDistStkDef'),
 (0.014726363881553803, 'red_SigBodyStkDef'),
 (0.014428429848460435, 'blue_StkGndAbsPerMin'),
 (0.014338643901345345, 'red_SigLegStkAcc'),
 (0.013052855176659474, 'red_SigDistStkAcc'),
 (0.011939148088596533, 'red_SigBodyStkAcc'),
 (0.011186246343307539, 'red_TDsPerMin'),
 (0.011055127530943667, 'red_StkLegPerMin'),
 (0.010906064752146219, 'red_sumSigClchSeen'),
 (0.010617940031263798, 'blue_StkLegAbsPerMin'),
 (0.009681810599264323, 'red_StkClchPerMin'),
 (0.00939291629446068, 'red_T

In [36]:
rf = RandomForestClassifier(n_estimators=105)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.618905472636816

In [37]:
preds = rf.predict(X)

In [38]:
sorted(zip(rf.feature_importances_, X), reverse=True)

[(0.0166794604372234, 'red_age'),
 (0.013727213807394952, 'blue_age'),
 (0.013370070530953588, 'blue_reachIN'),
 (0.012678470747125277, 'red_heightIN'),
 (0.01247549141908496, 'red_reachIN'),
 (0.01244669270715799, 'blue_heightIN'),
 (0.011086826285148703, 'red_StkDistAbsPerMin'),
 (0.010840948813052542, 'weight_class'),
 (0.01078802857477868, 'red_StkHeadAbsPerMin'),
 (0.010138149685618702, 'red_SigStkAbsPerMin'),
 (0.009881653939778181, 'red_TotStkAbsPerMin'),
 (0.0086163018671263, 'red_TDsPerMin'),
 (0.00823187329213022, 'red_StkBodyAbsPerMin'),
 (0.008150346634534709, 'red_SigStkPerMinDiff'),
 (0.007956380867552812, 'red_TotStkPerMinDiff'),
 (0.007934189503403112, 'red_SigDistStkDef'),
 (0.007913492736845204, 'red_SigStkDef'),
 (0.007833745991765164, 'red_sumSigHeadAbs'),
 (0.007774625126298673, 'blue_TotStkAcc'),
 (0.007329922465111631, 'red_SigDistStkAcc'),
 (0.0072652156813752195, 'red_StkHeadPerMin'),
 (0.0072496451586378284, 'red_SigHeadStkAcc'),
 (0.007136570714097468, 'red_S

In [39]:
# saving this model too, even though it preforms worse than our upsampled model to compare on future observations
filename = "../saved_models/rf_down_model.sav"
pickle.dump(rf, open(filename, "wb"))